# Run with trained model

In [1]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name="Tennis_Linux_NoVis/Tennis.x86_64")

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


In [2]:
import sys, platform, json
import torch
import random
import numpy as np

from scores import Scores
from ddpg_agent import Agent
from unityagents import UnityEnvironment

In [3]:
hparams = {
    "output": "default",
    "seed": 0,
    "epoch": 5000,
    "scores": {
        "expectation": 0.5,
        "window_size": 100,
        "check_solved": True
    },
    "t_max": 10000,
    "buffer_size":100000,
    "batch_size":256,
    "gamma":0.99,
    "tau":1e-3,
    "lr": { 
        "actor":1e-4,
        "critic":1e-4
    },
    "weight_decay": 0,

    "learn_per_step": 1,
    "update_times": 1,
    
    "hidden_layers": {
        "actor": [ 256, 128 ],
        "critic": [ 256, 128 ]
    }
}

In [4]:
# size of each action
action_size = brain.vector_action_space_size
states = env_info.vector_observations                  # get the current state (for each agent)
state_size = states.shape[1]

agent0 = Agent(state_size, action_size, hparams)
agent1 = Agent(state_size, action_size, hparams)

agent0.load('default_agent_0')
agent1.load('default_agent_1')

agents = [ agent0, agent1 ]

scores = []

In [5]:
scores = np.zeros(num_agents)
env_info = env.reset(train_mode=False)[brain_name]
states = env_info.vector_observations 
while True:
    actions = np.array([agents[i].act(states[i]) for i in range(num_agents)])

    env_info = env.step(actions)[brain_name]        # send the action to the environment
    next_states = env_info.vector_observations     # get the next state
    rewards = env_info.rewards                     # get the reward
    dones = env_info.local_done        

    states = next_states
    scores += rewards

    print('\rAgent0 : {:.2f}\tAgent1 : {:.2f}'.format(scores[0], scores[1]), end='') 

    if np.any(dones):
        break




Agent0 : 0.80	Agent1 : 0.79